#### Import Libraries
Import the sys and os libraries to manipulate the system path and the functions file containing the functions used to open the connection and query the database table. 

In [ ]:
import sys # Library for system-specific parameters and functions
import os # Library for interacting with the operating system
import pandas as pd

file = 'functions.py'
sys.path.insert(0,os.path.dirname(os.path.abspath(file)))
import functions # Custom functions for database operations

#### Set up Database Connection
Establish a connection to the PostgreSQL database using the **database_connection** function from the **functions** module. This function returns a connection object and a cursor object for executing SQL queries.

In [ ]:
# Enter the parameters from your database
# Do not expose the database credentials

host = 'host'
dbname = 'database_name'
user = 'user'

# Establish a connection to the database
connection, cursor = functions.database_connection(host = host, dbname = dbname, user = user)

#### Querying the main table
Some SQL query examples to demonstrate querying the main table of the database for specific information.

In [ ]:
# Example query 1:  Retrieve all records from the table main_table
#query_string = """SELECT * FROM main_table"""

# Example query 2: Retrieve records where crop is Wheat
query_string = """SELECT * FROM main_table WHERE crop = 'Rice'"""

# Example query 3: Retrieve records where crop is Wheat and measurement_method is Remote sensing
# query_string = """SELECT * FROM main_table WHERE crop = 'Wheat' AND measurement_method = 'Remote sensing'"""


# Example query 4: Retrieve records where crop is Wheat and measurement_method is Direct measurement or Proximal sensing
# query_string = """SELECT * FROM main_table WHERE crop = 'Wheat' 
#                         AND measurement_method = 'Direct measurement' 
#                         OR measurement_method = 'Proximal sensing'"""

# Example query 5: Retrieve records from columns variable_value and variable_units where crop is Wheat, treatment is Drought and season is Winter 2021-2022
# query_string = """SELECT variable_value, variable_units FROM main_table WHERE variable_name = 'Leaf SPAD-502 readings' 
#                         AND treatment =  'Drought'
#                         AND season = 'Winter 2021-2022'"""
# execute SQL query
cursor.execute(query_string)

# Fetch query results
query_results = cursor.fetchall()

# Create a DataFrame from query results
colnames = [desc[0] for desc in cursor.description]
data_frame = pd.DataFrame(data = query_results, columns = colnames)

# Sumary of the results
data_frame.info()

#### Querying the management table
Defines a SQL query string to select columns and rows from the table named management_data that meet the specified condition.

In [ ]:
# Example query 1:  Retrieve all records from the table management_data
query_string = """SELECT * FROM management_data"""

# execute SQL query
cursor.execute(query_string)

# Fetch query results
query_results = cursor.fetchall()

# Create a DataFrame from query results
colnames = [desc[0] for desc in cursor.description]
data_frame_mgmt = pd.DataFrame(data = query_results, columns = colnames)

# Sumary of the results
data_frame_mgmt.info()

#### Close the database conection
Close the cursor and the connection to the database to release resources.

In [ ]:
cursor.close()
connection.close()

#### Displaying dataframe
Displays the fetched data stored in the `data_frame` object. Data are ready for visualization and analysis!

In [ ]:
data_frame

#### Calculate Days After Seeding (DAS)
Extract dates for selected events from the `data_frame_mgmt` table, compute the difference in days relative to dates in the `data_frame` table, and add this difference as a new column.

In [ ]:
# Define the name of the day's difference column 
event_name = 'DAS' 

# define the event or list of events from the data_frame_mgmt table that are going to be used for the date extraction
selected_events = ['Seeds sown']

data_frame['date'] = pd.to_datetime(data_frame['date'])
data_frame_mgmt['date'] = pd.to_datetime(data_frame_mgmt['date'])

# Define a function to find the appropriate date from data_frame_dates
def find_matching_date(row):
    matching_row = data_frame_mgmt[
        (data_frame_mgmt['experiment'] == row['experiment']) &
        (data_frame_mgmt['crop'] == row['crop']) &
        (data_frame_mgmt['treatment'] == row['treatment']) &
        (data_frame_mgmt['season'] == row['season']) &
        (data_frame_mgmt['event'].isin(selected_events))
    ]
    return matching_row['date'].iloc[0] if not matching_row.empty else pd.NaT

# Apply the function to each row in data_frame to find the matching date
data_frame['matching_date'] = data_frame.apply(find_matching_date, axis=1)

# Calculate the difference in days and add it as a new column EVENT
data_frame[event_name] = (data_frame['date'] - data_frame['matching_date']).dt.days

# Drop the 'matching_date' column as it's no longer needed
data_frame.drop(columns=['matching_date'], inplace=True)